In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Estilo visual
sns.set(style="whitegrid", palette="pastel")

# Diccionario de preguntas
preguntas_dict = {
    "Horas de sueño (semana)": "¿Cuántas horas duermes en un día típico durante la semana?",
    "Horas de clases (semana)": "¿Cuántas horas tienes clases un día típico en la semana?",
    "Horas de estudio/tareas (semana)": "¿Cuántas horas estudias o haces tareas en un día típico en la semana?",
    "Horas extracurriculares (semana)": "¿Cuántas horas haces actividades extracurriculares en un día típico?",
    "Horas pantallas recreativas (semana)": "¿Cuántas horas usas pantallas recreativamente en un día típico en la semana?",
    "Horas descanso sin pantallas (semana)": "¿Cuántas horas descansas sin pantallas en un día típico en la semana?",
    "Horas de sueño (fin de semana)": "¿Cuántas horas duermes durante un día típico el fin de semana?",
    "Horas de estudio/tareas (fin de semana)": "¿Cuántas horas estudias o haces tareas durante un día típico el fin de semana?",
    "Horas pantallas recreativas (fin de semana)": "¿Cuántas horas usas pantallas recreativamente durante un día típico en el finde?",
    "Horas con familia o amistades (fin de semana)": "¿Cuántas horas compartes con familia o amistades durante un día típico el finde?",
    "Horas autocuidado (fin de semana)": "¿Cuántas horas dedicas a actividades físicas o de autocuidado durante un día típico el finde?"
}

# Orden lógico de cursos
orden = [
    "Tercero medio", "Cuarto medio",
    "Primer año de universidad", "Segundo año de universidad",
    "Tercer año de universidad", "Cuarto año de universidad",
    "Quinto año de universidad"
]

# --- INTERACCIÓN CON EL ESTUDIANTE DE COLEGIO ---
nombre_estudiante = input("Ingresa tu nombre: ").strip()

# Diccionario de respuestas
respuestas_estudiante = {}

for columna, pregunta in preguntas_dict.items():
    respuesta = input(f"{pregunta} ")
    try:
        respuestas_estudiante[columna] = float(respuesta)
    except:
        print("Entrada no válida. Se omitirá esta respuesta.")
        continue

# Crear DataFrame del estudiante de colegio
nuevo_df = pd.DataFrame({
    "Curso o año actual": [nombre_estudiante] * len(respuestas_estudiante),
    "Pregunta": list(respuestas_estudiante.keys()),
    "Horas": list(respuestas_estudiante.values())
})

# Reorganizar base universitaria
data_estudiantes = pd.read_csv('data_estudiantes_ingenieria.csv')
df_universitarios = data_estudiantes.copy()
df_universitarios = df_universitarios.melt(
    id_vars=["Curso o año actual"],
    value_vars=list(respuestas_estudiante.keys()),
    var_name="Pregunta",
    value_name="Horas"
)

# Filtrar solo universitarios para graficar (sin el estudiante)
df_universitarios = df_universitarios[
    df_universitarios["Curso o año actual"].isin(orden[2:])  # solo desde Primer año universidad
]

# Orden del eje X: cursos universitarios + nombre estudiante
orden_eje_x = orden[2:] + [nombre_estudiante]

# Agregar categoría ordenada
nuevo_df["Curso o año actual"] = pd.Categorical(
    nuevo_df["Curso o año actual"], categories=orden_eje_x, ordered=True
)
df_universitarios["Curso o año actual"] = pd.Categorical(
    df_universitarios["Curso o año actual"], categories=orden_eje_x, ordered=True
)

# Gráfico por cada pregunta
for columna in respuestas_estudiante:
    plt.figure(figsize=(13, 6))
    pregunta = preguntas_dict[columna]

    # Subset para gráfica
    subset = df_universitarios[df_universitarios["Pregunta"] == columna]

    # Dispersión de universitarios
    sns.stripplot(
        data=subset,
        x="Curso o año actual",
        y="Horas",
        hue="Curso o año actual",
        jitter=0.25,
        dodge=False,
        alpha=0.6,
        size=7,
        palette="Set2",
        legend=False  # evita duplicar leyendas
    )
    # Promedio de universitarios
    sns.pointplot(
        data=subset,
        x="Curso o año actual",
        y="Horas",
        estimator=np.mean,
        color="black",
        markers="D",
        linestyles="--",
        scale=1.5,
        errorbar=None
    )

    # Estrella del estudiante
    valor = respuestas_estudiante[columna]
    plt.scatter(
        [nombre_estudiante],
        [valor],
        color="red",
        marker="*",
        s=250
    )

    # Títulos y estilo
    plt.title(pregunta, fontsize=16, fontweight="bold")
    plt.xlabel("Curso o año actual", fontsize=12)
    plt.ylabel("Horas", fontsize=12)
    plt.xticks(rotation=35)
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.tight_layout()
    plt.show()
